In [89]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances

Data Loading

In [90]:
ratings_train = pd.read_csv("./Data/rate_train.csv", low_memory=False)
ratings_test = pd.read_csv("./Data/Ground_truth/groundtruth_0.3.csv",low_memory=False)

In [91]:
datasets_train = ratings_train.Node_Id.unique()
model_train = ratings_train.Model_Id.unique()
datasets_test = ratings_test.dataset.unique()
model_test = ratings_test.model.unique()
meta_models = pd.read_csv("./Data/model_v.csv",low_memory=False)
models = meta_models.model_id.unique()

Rating Matrix

In [92]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [93]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]


In [94]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [95]:
meta_datasets = pd.read_csv("./Data/dataset_v.csv",low_memory=False)
datasets = meta_datasets.dataset_id.unique()
meta_datasets = meta_datasets.loc[:,("v1","v2","v3","v4","v5")]
scaler = StandardScaler()

# 对dataframe的数据进行标准化
scaled_data = scaler.fit_transform(meta_datasets)
# 将标准化后的数据转换为dataframe，并保留原始索引
scaled_df = pd.DataFrame(scaled_data, index=meta_datasets.index, columns=meta_datasets.columns)
meta_dataset_similarity = cosine_similarity(scaled_df.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity,index=datasets,columns=datasets)

In [96]:
def calculate_user_similarity(users_ratings, user1, user2):
    """
    计算两个用户之间的相似度。

    参数:
    users_ratings: pandas.DataFrame，包含所有用户的评分记录，行为用户，列为电影。
    user1: str，第一个用户的名称。
    user2: str，第二个用户的名称。

    返回值:
    float，两个用户之间的相似度（值越大表示越相似）。
    """
    # 获取两个用户共同评价过的电影列表
    shared_movies = users_ratings.columns[(users_ratings.loc[user1].notna()) & (users_ratings.loc[user2].notna())]

    if len(shared_movies) == 0:
        # 如果两个用户没有共同评价的电影，则相似度为0
        return 0.0

    # 计算两个用户共同评价电影的评分
    user1_ratings = users_ratings.loc[user1, shared_movies]
    user2_ratings = users_ratings.loc[user2, shared_movies]

    # 计算皮尔逊相关系数
    mean_rating1 = user1_ratings.mean()
    mean_rating2 = user2_ratings.mean()
    numerator = np.sum((user1_ratings - mean_rating1) * (user2_ratings - mean_rating2))
    denominator = np.sqrt(np.sum((user1_ratings - mean_rating1) ** 2)) * np.sqrt(np.sum((user2_ratings - mean_rating2) ** 2))
    if denominator == 0:
        # 如果分母为0，则相似度为0
        return 0.0
    else:
        return numerator / denominator

In [97]:
dataset_similarity = pd.DataFrame(index=datasets_train,columns=datasets_train)

In [98]:
start_time_train = time.time()

In [99]:
lambda_ = 0.5
for i in datasets_train:
    for j in datasets_train:
        rating_based_sim = calculate_user_similarity(data_model_train_matrix,i,j)
        if rating_based_sim!= 0:
            dataset_similarity.loc[i][j] = rating_based_sim
        else:
            dataset_similarity.loc[i][j] = 0


In [100]:
dataset_similarity

,0,29,14,28,16,17,13,12,10,38,...,275,274,279,278,280,281,288,286,287,285
0,1.0,-0.050256,-0.001617,0.01316,-0.100441,0.155723,0.073391,0.103242,-0.008401,0.019664,...,-0.774856,-0.725499,-0.826643,-0.574186,-0.471846,-0.753677,0.991066,0.988679,0.987632,0.991627
29,-0.050256,1.0,0.338583,0.343427,0.303728,0.756781,0.787525,0.772851,0.829523,0.930355,...,0.127344,0.168294,0.158413,0.161688,0.148418,0.162801,-0.053548,-0.052812,-0.053901,-0.054478
14,-0.001617,0.338583,1.0,0.882878,0.896983,0.318854,0.334677,0.328099,0.434245,0.355652,...,0.320345,0.33979,0.337513,0.30064,0.285865,0.358922,-0.031076,-0.042566,-0.041411,-0.034546
28,0.01316,0.343427,0.882878,1.0,0.914768,0.338096,0.319927,0.313646,0.392878,0.34945,...,0.303083,0.290097,0.251706,0.308339,0.337615,0.341945,-0.018841,-0.026223,-0.02447,-0.023382
16,-0.100441,0.303728,0.896983,0.914768,1.0,0.317998,0.318539,0.314436,0.38817,0.313943,...,0.413201,0.354929,0.368646,0.338358,0.321499,0.421779,-0.119706,-0.128368,-0.125522,-0.126305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,-0.753677,0.162801,0.358922,0.341945,0.421779,-0.001343,0.047578,0.017444,0.127751,0.117882,...,0.761404,0.777061,0.863,0.715754,0.790664,1.0,-0.759594,-0.763402,-0.766307,-0.76131
288,0.991066,-0.053548,-0.031076,-0.018841,-0.119706,0.142879,0.067142,0.0976,-0.016817,0.012564,...,-0.771086,-0.725171,-0.820865,-0.575953,-0.495113,-0.759594,1.0,0.998672,0.998006,0.999399
286,0.988679,-0.052812,-0.042566,-0.026223,-0.128368,0.138789,0.062577,0.092768,-0.020848,0.011939,...,-0.76673,-0.720674,-0.8163,-0.57604,-0.4961,-0.763402,0.998672,1.0,0.999616,0.997821
287,0.987632,-0.053901,-0.041411,-0.02447,-0.125522,0.13917,0.063274,0.095206,-0.022511,0.011341,...,-0.764251,-0.72166,-0.81356,-0.581117,-0.503278,-0.766307,0.998006,0.999616,1.0,0.99682


Prediction

In [101]:
def predict(rating_matrix, similarity_matrix):
    """
    根据评分矩阵和相似度矩阵预测评分。

    参数：
    rating_matrix (pd.DataFrame)：评分矩阵，包含NaN值
    similarity_matrix (numpy.array)：相似度矩阵

    返回：
    pd.DataFrame：预测评分矩阵
    """

    # 获取评分矩阵的均值（忽略NaN值）
    mean_rating = rating_matrix.mean(axis=1).values

    # 将评分矩阵中的NaN值替换为0
    rating_matrix_nan_to_zero = rating_matrix.fillna(0).values

    # 减去均值，得到归一化的评分矩阵
    normalized_rating_matrix = rating_matrix_nan_to_zero - mean_rating[:, np.newaxis]

    # 计算预测评分
    predicted_ratings = mean_rating[:, np.newaxis] + np.dot(similarity_matrix, normalized_rating_matrix) / np.abs(similarity_matrix).sum(axis=1)[:, np.newaxis]

    # 将预测评分数组转换为DataFrame
    predicted_ratings_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

    return predicted_ratings_df

In [102]:
def predict_ratings(rating_matrix, user_similarity_matrix,k):
    """
    输入：
    rating_matrix - 评分矩阵，DataFrame格式，其中NaN表示未评分
    user_similarity_matrix - 用户相似度矩阵，DataFrame格式
    k - 最近邻的数量，默认为5

    输出：
    prediction_matrix - 预测矩阵，DataFrame格式
    """

    # 初始化预测矩阵
    prediction_matrix = rating_matrix.copy()

    # 对于评分矩阵中的每个NaN值，使用K最近邻的方法预测评分
    for i in rating_matrix.index:
        for j in rating_matrix.columns:
            if np.isnan(rating_matrix.loc[i][j]):
                # 获取第i个用户的相似度值，并在相似度矩阵中找到K个最相似的用户
                similarity_values = user_similarity_matrix.loc[i].sort_values(ascending=False)[1:k+1]

                # 计算加权平均评分
                weighted_sum = 0
                similarity_sum = 0
                for index, value in similarity_values.items():
                    user_rating = rating_matrix.loc[index][j]
                    if not np.isnan(user_rating):
                        weighted_sum += value * user_rating
                        similarity_sum += value

                # 如果存在至少一个相似用户对该物品进行了评分，则计算预测评分
                if similarity_sum != 0:
                    prediction_matrix.loc[i][j] = weighted_sum / similarity_sum
                else:
                    # 如果没有相似用户评分，则使用当前商品的平均评分作为预测值
                    prediction_matrix.loc[i][j] = rating_matrix[j].mean()

    return prediction_matrix

In [103]:
model_prediction_train = predict(data_model_train_matrix,dataset_similarity)

c:\users\byy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


In [104]:
model_prediction_train = pd.DataFrame(model_prediction_train,index=datasets_train,columns=models).sort_index().sort_index(axis=1)

In [105]:
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

In [106]:
end_time_train = time.time()

In [107]:
Training_time = end_time_train - start_time_train
Training_time

75.74504041671753

Metadata Based

In [133]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [146]:
def Find_Top_k(i,sim_matrix):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row

In [147]:
start_time = time.time()

In [148]:
meta_dataset_similarity

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,288
0,1.000000,-0.185879,-0.185879,-0.185879,-0.185879,-0.185879,-0.185879,-0.185879,-0.185879,-0.185879,...,-0.248303,-0.342076,-0.342076,-0.342076,-0.247496,-0.247496,0.633129,0.633129,0.633129,0.633129
1,-0.185879,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.413583,-0.302929,-0.302929,-0.302929,-0.156554,-0.156554,0.045438,0.045438,0.045438,0.045438
2,-0.185879,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.413583,-0.302929,-0.302929,-0.302929,-0.156554,-0.156554,0.045438,0.045438,0.045438,0.045438
3,-0.185879,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.413583,-0.302929,-0.302929,-0.302929,-0.156554,-0.156554,0.045438,0.045438,0.045438,0.045438
4,-0.185879,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.413583,-0.302929,-0.302929,-0.302929,-0.156554,-0.156554,0.045438,0.045438,0.045438,0.045438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,-0.247496,-0.156554,-0.156554,-0.156554,-0.156554,-0.156554,-0.156554,-0.156554,-0.156554,-0.156554,...,0.700951,0.846408,0.846408,0.846408,1.000000,1.000000,0.455987,0.455987,0.455987,0.455987
285,0.633129,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,...,-0.039051,0.066057,0.066057,0.066057,0.455987,0.455987,1.000000,1.000000,1.000000,1.000000
286,0.633129,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,...,-0.039051,0.066057,0.066057,0.066057,0.455987,0.455987,1.000000,1.000000,1.000000,1.000000
287,0.633129,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,0.045438,...,-0.039051,0.066057,0.066057,0.066057,0.455987,0.455987,1.000000,1.000000,1.000000,1.000000


In [149]:
for dataset in datasets_test:
    for model in model_test:
        dataset_sim_list = Find_Top_k(dataset,meta_dataset_similarity)[1:]
        # 仅保留存在于 model_prediction_train 的索引
        valid_indices = [idx for idx in dataset_sim_list if idx in model_prediction_train.index][:15]
        model_prediction_test.loc[dataset][model] = model_prediction_train.loc[valid_indices][model].mean()

In [150]:
valid_indices

[271, 270, 269, 281, 280, 266, 265, 264, 263, 262, 255, 256, 268, 276, 279]

In [151]:
end_time = time.time()

In [152]:
end_time - start_time

9.618462800979614

In [153]:
for i in datasets_test:
    for j in model_test:
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [154]:
result = pd.DataFrame(columns={"dataset","model","f1_score","groundtruth_f1_score"})

In [155]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            f1_score = model_prediction_test.loc[i][j]
            groundtruth_f1_score = data_model_test_matrix.loc[i][j]
            result = result.append([{'dataset':i,'model':j,'f1_score':f1_score,'groundtruth_f1_score':groundtruth_f1_score}],ignore_index=True)

In [156]:
result

,model,f1_score,groundtruth_f1_score,dataset
0,463,0.377049,0.300000,93
1,465,0.372282,0.300000,93
2,363,0.545290,0.909091,93
3,367,0.573518,0.303030,93
4,369,0.561251,0.303030,93
...,...,...,...,...
8207,307,0.427334,0.908689,283
8208,317,0.347039,0.882246,283
8209,315,0.346212,0.885276,283
8210,303,0.376006,0.751981,283


In [157]:
result.to_csv("../Peak_Finding/Output/Rating_only/Full_Rating_only@15.csv",index=False)